In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import scipy 
import sklearn
from collections import Counter
from sklearn.metrics import multilabel_confusion_matrix
from scipy import spatial
from sklearn.model_selection import train_test_split

In [2]:
read_data = pd.read_csv('Datasets/kmeans_data/data.csv')
read_labels = pd.read_csv('Datasets/kmeans_data/label.csv',names=['label'],header=None)

In [3]:
read_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Columns: 784 entries, 0 to 0.667
dtypes: int64(784)
memory usage: 59.8 MB


In [4]:
read_data.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
train_data, test_data = train_test_split( read_data, test_size=0.08, random_state=42)
train_labels, test_labels = train_test_split( read_labels, test_size=0.08, random_state=42)

In [6]:
class Kmeans_algo:
    
    def sseCalculation(self, centr_value_dict, centr_dict,read_data):
        sse_data = 0
        for i in centr_dict:
            sse_cluster = 0
            # np.sum()
            for j in centr_dict[i]:
                d = list(read_data.iloc[int(j)])
                for a,b in zip(centr_value_dict[i],d):
                    sse_cluster += (a-b)**2
            sse_data+=sse_cluster
        return sse_data    
    
    def initializingCent(self,read_data,K):
        m = read_data.shape[0]
        centr_value_dict={}
        for i in range(K):
            r = np.random.randint(0, m-1)
            centr_value_dict[i] = read_data.iloc[r]
        return centr_value_dict
    
    def jaccardSimilarity(self,centr, d):
        intersection = len(list(set(centr).intersection(d)))
        union = (len(set(centr)) + len(set(d))) - intersection
        return float(intersection) / union

    def trainKmeans(self,read_data,K,max_iter=20,mode=1,tol=10):
        centr_value_dict = self.initializingCent(read_data,K)
        new_centr_value_dict = {}
        count = 0
        centr_dict = {}
        convergence = False
        while((count<max_iter) and not convergence):
            
            for i in list(centr_value_dict.keys()):
                centr_dict[i]=[]
            for i in range(read_data.shape[0]):
                x = read_data.iloc[i]
                if mode==1 :
                    distance_measure = [np.linalg.norm(x-centr_value_dict[j])  for j in centr_value_dict]
                    idx = np.argmin(distance_measure)
                    centr_dict[idx].append(i)
                elif mode==2 :
                    distance_measure = [self.jaccardSimilarity(list(x),centr_value_dict[j]) for j in centr_value_dict]
                    idx = np.argmax(distance_measure)
                    centr_dict[idx].append(i)
                elif mode==3 :
                    distance_measure = [1-scipy.spatial.distance.cosine(x,list(centr_value_dict[j]))  for j in centr_value_dict]
                    idx = np.argmax(distance_measure)
                    centr_dict[idx].append(i)
                
                prev_centr=dict(centr_value_dict)
                
            
            for i in centr_dict:
                if len(centr_dict[i]):
                    dps_centr = centr_dict[i]
                    centr_value_dict[i] = np.average(read_data.iloc[dps_centr],axis=0)
            
            
            current_tol=-1
            for i in centr_value_dict:
                prev_centr_point = prev_centr[i]
                new_centr_point = centr_value_dict[i]
                change = np.sum(np.absolute(new_centr_point-prev_centr_point))
                current_tol = max(change, current_tol)
            print("SSE value for iteration ",count,": ", self.sseCalculation(centr_value_dict, centr_dict, read_data)) 
            print("Tolerance for the Iteration ",count,": ",current_tol)
            
            count+=1
            if (current_tol<10):
                convergence = True
                break
           # print("KMeans Iteration",count)
        return centr_value_dict,centr_dict

In [7]:
def predict_cluster_labels(C, S, labels):
    cluster_labels = np.zeros(10,dtype=int)
    for c in C:
        labels_of_points = []
        for point in S[c]:
            labels_of_points.extend(labels.iloc[point])
        counter = Counter(labels_of_points)
        try:
            cluster_labels[c] = max(counter, key=counter.get)
        except:
            cluster_labels[c] = np.random.randint(0,9)
    return cluster_labels

In [8]:
def accuracy(centroids, centroid_Labels, test_data, true_labels, mode=1):
    y_true = list(true_labels['label']);
    y_pred = []
    for index in range(test_data.shape[0]):
        featureset = test_data.iloc[index]
        if mode==1:
            distances = [np.linalg.norm(featureset - centroids[centroid]) for centroid in centroids]
            classification = distances.index(min(distances))
            y_pred.append(centroid_Labels[classification])
        elif mode==2:
            similarity = [jaccardSimilarity(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification]) 
        elif mode==3:
            similarity = [1 - spatial.distance.cosine(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification])
    denominator = test_data.shape[0]
    correctly_classified = 0
    for i in range(0,len(y_pred)):
        if y_true[i] == y_pred[i]:
            correctly_classified += 1
    accuracy = correctly_classified/denominator
    return accuracy


In [9]:
kmeans_model = Kmeans_algo()
centroids_euclidian,clusters_euclidian = kmeans_model.trainKmeans(read_data,10, max_iter=100,mode=1)

SSE value for iteration  0 :  29063767116.12679
Tolerance for the Iteration  0 :  22744.890595009598
SSE value for iteration  1 :  26987660351.83455
Tolerance for the Iteration  1 :  7399.941705990486
SSE value for iteration  2 :  26304921310.92179
Tolerance for the Iteration  2 :  4918.691764751887
SSE value for iteration  3 :  25948279368.959244
Tolerance for the Iteration  3 :  3740.0271179549063
SSE value for iteration  4 :  25757219310.772552
Tolerance for the Iteration  4 :  2621.5272593952286
SSE value for iteration  5 :  25654062849.206013
Tolerance for the Iteration  5 :  2041.4604639010468
SSE value for iteration  6 :  25586447229.2848
Tolerance for the Iteration  6 :  1844.6750666249945
SSE value for iteration  7 :  25538689598.864914
Tolerance for the Iteration  7 :  1574.3413228346458
SSE value for iteration  8 :  25504563649.252552
Tolerance for the Iteration  8 :  1395.270811522395
SSE value for iteration  9 :  25483420382.255672
Tolerance for the Iteration  9 :  934.822

In [10]:
SSE_Euclidean = kmeans_model.sseCalculation(centroids_euclidian,clusters_euclidian,read_data)
print("SSE value for Euclidean = ",SSE_Euclidean)

SSE value for Euclidean =  25372068803.044205


In [11]:
cluster_labels_euclidian = predict_cluster_labels(centroids_euclidian,clusters_euclidian,read_labels)
cluster_labels_euclidian

array([3, 5, 7, 3, 9, 9, 0, 0, 1, 2])

In [12]:
Euclidean_accuracy = accuracy(centroids_euclidian, cluster_labels_euclidian,test_data,test_labels)
Euclidean_accuracy

0.09375

In [13]:
centroids_jaccard,clusters_jaccard = kmeans_model.trainKmeans(read_data,10, max_iter=100,mode=2)
SSE_jaccard = kmeans_model.sseCalculation(centroids_jaccard,clusters_jaccard,read_data)

SSE value for iteration  0 :  34144779471.069885
Tolerance for the Iteration  0 :  35792.27618433774
SSE value for iteration  1 :  34361687572.938736
Tolerance for the Iteration  1 :  26323.993799379936
SSE value for iteration  2 :  34305153676.593925
Tolerance for the Iteration  2 :  12115.617141511655
SSE value for iteration  3 :  34341364147.999336
Tolerance for the Iteration  3 :  5233.925366300367
SSE value for iteration  4 :  34338992007.305153
Tolerance for the Iteration  4 :  2383.03672669309
SSE value for iteration  5 :  34347476778.461205
Tolerance for the Iteration  5 :  2151.7423185581138
SSE value for iteration  6 :  34331654269.928753
Tolerance for the Iteration  6 :  927.982295578079
SSE value for iteration  7 :  34361687572.938736
Tolerance for the Iteration  7 :  659.835333697125
SSE value for iteration  8 :  34361687572.938736
Tolerance for the Iteration  8 :  1593.3019194996423
SSE value for iteration  9 :  34361687572.938736
Tolerance for the Iteration  9 :  802.607

In [99]:
print("SSE value for Jaccard  = ",SSE_jaccard)

SSE value for Jaccard  =  34361687572.938736


In [14]:
cluster_labels_jaccard = predict_cluster_labels(centroids_jaccard,clusters_jaccard,read_labels)
cluster_labels_jaccard

array([1, 8, 5, 8, 1, 0, 0, 7, 7, 8])

In [15]:
Accuracy_Jaccard = accuracy(centroids_jaccard, cluster_labels_jaccard,test_data,test_labels)
Accuracy_Jaccard

0.1075

In [16]:
centroids_cossine,clusters_cossine = kmeans_model.trainKmeans(read_data,10, max_iter=100,mode=3)
SSE_cossine = kmeans_model.sseCalculation(centroids_cossine,clusters_cossine,read_data)

SSE value for iteration  0 :  28143892301.059044
Tolerance for the Iteration  0 :  28293.965601965603
SSE value for iteration  1 :  26453270584.375805
Tolerance for the Iteration  1 :  4661.702241199839
SSE value for iteration  2 :  26064800363.76714
Tolerance for the Iteration  2 :  3599.680484587543
SSE value for iteration  3 :  25912363262.45425
Tolerance for the Iteration  3 :  2044.560350455972
SSE value for iteration  4 :  25859210064.426548
Tolerance for the Iteration  4 :  1050.6214454576157
SSE value for iteration  5 :  25826378199.19674
Tolerance for the Iteration  5 :  1132.7750539319209
SSE value for iteration  6 :  25788593523.325306
Tolerance for the Iteration  6 :  1561.5035167335068
SSE value for iteration  7 :  25749791488.5029
Tolerance for the Iteration  7 :  1640.7930207600932
SSE value for iteration  8 :  25710589625.14729
Tolerance for the Iteration  8 :  1558.319980719134
SSE value for iteration  9 :  25679297737.328876
Tolerance for the Iteration  9 :  1536.0777

In [17]:
print("SSE value for cossine  = ",SSE_cossine)

SSE value for cossine  =  25553949716.612648


In [18]:
cluster_labels_cossine = predict_cluster_labels(centroids_cossine,clusters_cossine,read_labels)
cluster_labels_cossine

array([3, 4, 1, 8, 9, 2, 0, 7, 3, 0])

In [19]:
Accuracy_cossine = accuracy(centroids_cossine, cluster_labels_cossine,test_data,test_labels)
Accuracy_cossine

0.0725

In [20]:
print("Euclidean accuracy = ",Euclidean_accuracy)
print("Jacard accuracy = ",Accuracy_Jaccard)
print("Cosine accuracy = ",Accuracy_cossine)

Euclidean accuracy =  0.09375
Jacard accuracy =  0.1075
Cosine accuracy =  0.0725


In [21]:
print("SSE value for Euclidean = ",SSE_Euclidean)
print("SSE value for Jaccard = ",SSE_jaccard)
print("SSE value for cossine  = ",SSE_cossine)

SSE value for Euclidean =  25372068803.044205
SSE value for Jaccard =  34361687572.938736
SSE value for cossine  =  25553949716.612648
